# GDMA - Assignment 3

Author: Julian Schelb (1069967)

In [1]:
from neo4j import GraphDatabase
import matplotlib.pyplot as plt
import pandas as pd

### Connection to the database instance

In [2]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "subatomic-shrank-Respond"))
session = driver.session()

### Question 1

Import the graph from the attached karate.csv file. Then, using functions from the Graph Data Science Library (GDS), write Cypher queries to compute the Betweenness, Closeness, and Eigenvector centrality of each
node. You can find the full description of the functions at:

https://neo4j.com/docs/graph-data-science/current/algorithms/centrality/

**Delete Existing Data:**

In [3]:
query = """
match (a) delete a
"""

with driver.session() as session:
    result = session.run(query)
    
    
query = """
match (a) -[r] -> () delete a, r
"""

with driver.session() as session:
    result = session.run(query)
    

**Import Data:**

In [4]:
query = """
LOAD CSV FROM 'file:///karate.csv' AS row FIELDTERMINATOR ';'
WITH row[0] as sourceId, row[1] as targetId
MERGE (s:Node {id: sourceId})
MERGE (t:Node {id: targetId})
MERGE (s)-[:RELATED_TO]->(t)   
RETURN s, t
"""

with driver.session() as session:
    result = session.run(query)

**Create In-Memory Graph Projection:**

In [5]:
query = """
CALL gds.graph.drop('tmpGraph', false) 
"""

with driver.session() as session:
    result = session.run(query)
    
query = """
CALL gds.graph.project('tmpGraph', 'Node', 'RELATED_TO')
"""

with driver.session() as session:
    result = session.run(query)

**Betweenness Centrality:**

In [6]:
query = """
CALL gds.betweenness.stream('tmpGraph')
YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS nodeId, score
MATCH (n:Node) 
WHERE  n.id = nodeId
SET n.betweenness = score
RETURN n.id, score
ORDER BY score DESC
"""

dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head()

,n.id,score
0,3,8.833333
1,32,5.083333
2,9,2.250000
3,29,2.166667
4,4,2.000000


**Eigenvector Centrality:**

In [7]:
query = """
CALL gds.eigenvector.stream('tmpGraph')
YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS nodeId, score
MATCH (n:Node) 
WHERE  n.id = nodeId
SET n.eigenvector = score
RETURN n.id, score
ORDER BY score DESC
"""
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head()

,n.id,score
0,34,0.993455
1,33,0.113378
2,14,0.006594
3,8,0.006594
4,13,0.006138


**Closeness Centrality:**

In [8]:
query = """
CALL gds.beta.closeness.stream('tmpGraph')
YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS nodeId, score
MATCH (n:Node) 
WHERE  n.id = nodeId
SET n.closeness = score
RETURN n.id, score
ORDER BY score DESC
"""
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(20)

,n.id,score
0,30,1.000000
1,14,1.000000
2,20,1.000000
3,11,1.000000
4,12,1.000000
5,18,1.000000
6,2,1.000000
7,3,1.000000
8,5,1.000000
9,6,1.000000


**Result:**

In [9]:
query = """
MATCH (a)
RETURN a.id, a.betweenness, a.eigenvector, a.closeness
"""        
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data

,a.id,a.betweenness,a.eigenvector,a.closeness
0,15,0.000000,8.660452e-09,0.000000
1,16,0.000000,8.660452e-09,0.000000
2,19,0.000000,8.660452e-09,0.000000
3,21,0.000000,8.660452e-09,0.000000
4,23,0.000000,8.660452e-09,0.000000
5,30,1.000000,1.282623e-06,1.000000
6,31,0.833333,5.744748e-03,0.666667
7,32,5.083333,5.746022e-03,0.636364
8,10,0.166667,4.348725e-04,0.600000
9,14,1.750000,6.593799e-03,1.000000


***

Finally, write a Cypher query that removes the node from the graph with
the highest centrality only if the graph is weakly connected. The check
whether the graph is weakly connected must be done with Cypher as well.
Also, write a python function that executes the Cypher query repeatedly
until the graph becomes disconnected. Ideally, the python code must
execute only a single Cypher query per iteration.

**Determinig node with highest centrality score:**

In [10]:
query = """
MATCH (n)
WITH n as node, n.eigenvector as centrality
WITH node, centrality
ORDER BY centrality DESC
LIMIT 1
RETURN *
//DETACH DELETE node
"""        
#dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
#dtf_data

**Check if graph is weakly connected:**

In [11]:
query = """
CALL gds.wcc.stream('tmpGraph')
YIELD nodeId, componentId
WITH size(collect(DISTINCT componentId)) as component_count
RETURN 
  CASE component_count
  WHEN 1 
  THEN true 
  ELSE false 
  END AS result
  
"""        
##dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
#dtf_data

**Combined Soltution:**

_Assumption:_ Centrality is supposed to be recalculated during every iteration.

In [12]:
query = """

// Update graph projection
CALL gds.graph.drop('tmpGraph', false) 
YIELD graphName as graph_dropped
CALL gds.graph.project('tmpGraph', 'Node', 'RELATED_TO')
YIELD graphName as graph_created

// Determine count of weakly connected components
CALL gds.wcc.stream('tmpGraph')
YIELD nodeId, componentId
WITH size(collect(DISTINCT componentId)) as component_count
WITH CASE component_count WHEN 1 THEN true ELSE false END AS weakly_connected

// (Re-)calculate centrality score
CALL gds.eigenvector.stream('tmpGraph')
YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS nodeId, score, weakly_connected

// Find nodes with highest centrality
MATCH (n:Node)
WHERE weakly_connected AND n.id = nodeId
WITH n as node, score as centrality, weakly_connected
WITH node, node.id as nodeId , centrality, weakly_connected
ORDER BY centrality DESC
LIMIT 1

// Delete node
DETACH DELETE node
RETURN nodeId, weakly_connected

"""

weakly_connected = True
iter = 0

while weakly_connected:
    iter = iter + 1
    with driver.session() as session:
        result = session.run(query)
        
        i = 0
        for record in result:
            print(f"Iteration: {iter}: Node to remove: {record['nodeId']}, weakly conntected: {record['weakly_connected']}")
            i = i + 1
         
    if i == 0: weakly_connected = False
    
    
print(f"Stopped after removing {iter - 1} nodes.")

Iteration: 1: Node to remove: 34, weakly conntected: True
Iteration: 2: Node to remove: 33, weakly conntected: True
Stopped after removing 2 nodes.


**Original Graph:**

svg image

**After removing node 34 in iteration 1:**

svg image

**After removing node 33 in iteration 2:**

svg image

### Question 2

The two files 0.edges and 348.edges represent two different graphs G1
and G2. Load both graphs into a single database. The result should
be a large disconnected graph. Then compute the betweenness centrality
distributions for both G1 and G2 (basically you need store the betweenness
centralities in two separate lists). Finally, compute the similarity of the
two distributions using any applicable similarity provided by GDS. You
can find the descriptions of the functions at:

https://neo4j.com/docs/graph-data-science/current/algorithms/similarity-functions/

**Import Data:**

In [13]:
query = """
DROP DATABASE sec IF EXISTS
"""

with driver.session() as session:
    result = session.run(query)
    
query = """
CREATE DATABASE sec IF NOT EXISTS
"""

with driver.session() as session:
    result = session.run(query)

In [14]:
query = """
LOAD CSV FROM 'file:///0edges.sec' AS row FIELDTERMINATOR ' '
WITH row[0] as sourceId, row[1] as targetId
MERGE (s:NodeG1 {id: sourceId, graph: 1})
MERGE (t:NodeG1 {id: targetId, graph: 1})
MERGE (s)-[:RELATED_TO]->(t)   
RETURN s, t
"""

with driver.session(database = "sec") as session:
    result = session.run(query)

In [15]:
query = """
LOAD CSV FROM 'file:///348edges.sec' AS row FIELDTERMINATOR ' '
WITH row[0] as sourceId, row[1] as targetId
MERGE (s:NodeG2 {id: sourceId, graph: 2})
MERGE (t:NodeG2 {id: targetId, graph: 2})
MERGE (s)-[:RELATED_TO]->(t)   
RETURN s, t
"""

with driver.session(database = "sec") as session:
    result = session.run(query)

**Create In-Memory Graph Projection:**

In [16]:
query = """
CALL gds.graph.drop('secGraph', false) 
"""

with driver.session(database = "sec") as session:
    result = session.run(query)
    
query = """
CALL gds.graph.project('secGraph',  ['NodeG1', 'NodeG2'], 'RELATED_TO')
"""

with driver.session(database = "sec") as session:
    result = session.run(query)

**Compare Centrality Distributions:**

In [17]:
query = """
// Centrality Graph 1
CALL gds.eigenvector.stream('secGraph', {nodeLabels: ['NodeG1']})
YIELD nodeId as nodeId_G1, score as scores_G1
WITH collect(scores_G1) as scores_G1

// Centrality Graph 2
CALL gds.eigenvector.stream('secGraph', {nodeLabels: ['NodeG2']})
YIELD nodeId as nodeId_G2, score as scores_G2
WITH collect(scores_G2) as scores_G2, scores_G1

//Padding smaller Distribution
WITH 
scores_G1 + [x IN range(1,size(scores_G2) - size(scores_G1))| 0] AS scores_G1,
scores_G2 + [x IN range(1,size(scores_G1) - size(scores_G2))| 0] AS scores_G2

//Calculate Distance
RETURN 
//size(scores_G1) as len_G1,
//size(scores_G2) as len_G2,
gds.similarity.pearson(scores_G1, scores_G2) as pearsonSimilarity,
gds.similarity.overlap(scores_G1, scores_G2) as overlapSimilarity,
gds.similarity.jaccard(scores_G1, scores_G2) as jaccardSimilarity,
gds.similarity.cosine(scores_G1, scores_G2) AS cosineSimilarity,
gds.similarity.euclidean(scores_G1,scores_G2) AS euclideanSimilarity,
gds.similarity.euclideanDistance(scores_G1,scores_G2) AS euclideanDistance
"""

dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data


,pearsonSimilarity,overlapSimilarity,jaccardSimilarity,cosineSimilarity,euclideanSimilarity,euclideanDistance
0,0.396912,0.0,0.0,0.590634,0.524979,0.904838
